### Best score is coming at missing -999, non-normalised, > 96% dropping features : 66% something

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD, PCA, FastICA
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import make_pipeline, make_union
from sklearn.manifold import TSNE

In [2]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 10000)

In [3]:
with open('train_iI26erW/train.csv', 'r') as fp, open('test_plBmD8c.csv') as fg:
    train = pd.read_csv(fp)
    test = pd.read_csv(fg)

/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,11,12,13,14,15,16,17,18,19,21,23,26,28,29,30,32,82,83,85,91,269,325,326,327,328) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,11,12,13,14,15,16,17,18,19,26,28,29,30,82,83,85,91,269,325,326,327,328) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
print(train.shape)
print(test.shape)

(300000, 333)
(200000, 332)


In [5]:
## set blank as na
train = train.replace('', np.nan)
test = test.replace('', np.nan)

In [6]:
## count no. of NA per row

train['na_count'] = train.isnull().sum(axis=1)
test['na_count'] = test.isnull().sum(axis=1)

In [7]:
## net balance after spending in mon 1
addCols = [x for x in train.columns if 'CARD_' in x]
train['add_spend_mon1'] = train[addCols].sum(axis = 1) 
test['add_spend_mon1'] = test[addCols].sum(axis = 1)

train['add_spend_mon1'] = train['add_spend_mon1'].map(lambda x: x.count('Y'))
test['add_spend_mon1'] = test['add_spend_mon1'].map(lambda x: x.count('Y'))

In [8]:
train['CLOSED_DATE'] = train['CLOSED_DATE'].astype(str)
test['CLOSED_DATE'] = test['CLOSED_DATE'].astype(str)

In [9]:
def getDateFormat(val):
    if not val == 'nan':
        try:
            a = datetime.strptime(val, '%d%b%Y')
            b = a.strftime('%Y-%m-%d')
            return b
        except TypeError:
            return np.nan
    else:
        return np.nan

In [10]:
from datetime import datetime

train['closed_date_format'] = train['CLOSED_DATE'].map(getDateFormat)
test['closed_date_format'] = test['CLOSED_DATE'].map(getDateFormat)

train['maturity_gl_format'] = train['MATURITY_GL'].map(getDateFormat)
test['maturity_gl_format'] = test['MATURITY_GL'].map(getDateFormat)

train['maturity_lap_format'] = train['MATURITY_LAP'].map(getDateFormat)
test['maturity_lap_format'] = test['MATURITY_LAP'].map(getDateFormat)

train['maturity_las_format'] = train['MATURITY_LAS'].map(getDateFormat)
test['maturity_las_format'] = test['MATURITY_LAS'].map(getDateFormat)

In [11]:
## drop date formats
train.drop(['CLOSED_DATE','MATURITY_GL','MATURITY_LAP','MATURITY_LAS'], axis=1, inplace=True)
test.drop(['CLOSED_DATE','MATURITY_GL','MATURITY_LAP','MATURITY_LAS'], axis=1, inplace=True)

In [12]:
## date features

train['closed_date_format'] = pd.to_datetime(train['closed_date_format'])
test['closed_date_format'] = pd.to_datetime(test['closed_date_format'])

train['maturity_gl_format'] = pd.to_datetime(train['maturity_gl_format'])
test['maturity_gl_format'] = pd.to_datetime(test['maturity_gl_format'])

train['maturity_lap_format'] = pd.to_datetime(train['maturity_lap_format'])
test['maturity_lap_format'] = pd.to_datetime(test['maturity_lap_format'])

train['maturity_las_format'] = pd.to_datetime(train['maturity_las_format'])
test['maturity_las_format'] = pd.to_datetime(test['maturity_las_format'])

In [13]:
train['closed_date_weekday'] = train['closed_date_format'].dt.weekday
test['closed_date_weekday'] = test['closed_date_format'].dt.weekday

train['closed_date_year'] = train['closed_date_format'].dt.year
test['closed_date_year'] = test['closed_date_format'].dt.year

train['closed_date_month'] = train['closed_date_format'].dt.month
test['closed_date_month'] = test['closed_date_format'].dt.month

train['closed_date_day'] = train['closed_date_format'].dt.day
test['closed_date_day'] = test['closed_date_format'].dt.day

#########################################################

train['maturity_gl_weekday'] = train['maturity_gl_format'].dt.weekday
test['maturity_gl_weekday'] = test['maturity_gl_format'].dt.weekday

train['maturity_gl_year'] = train['maturity_gl_format'].dt.year
test['maturity_gl_year'] = test['maturity_gl_format'].dt.year

train['maturity_gl_month'] = train['maturity_gl_format'].dt.month
test['maturity_gl_month'] = test['maturity_gl_format'].dt.month

train['maturity_gl_day'] = train['maturity_gl_format'].dt.day
test['maturity_gl_day'] = test['maturity_gl_format'].dt.day

################################################################

train['maturity_lap_weekday'] = train['maturity_lap_format'].dt.weekday
test['maturity_lap_weekday'] = test['maturity_lap_format'].dt.weekday

train['maturity_lap_year'] = train['maturity_lap_format'].dt.year
test['maturity_lap_year'] = test['maturity_lap_format'].dt.year

train['maturity_lap_month'] = train['maturity_lap_format'].dt.month
test['maturity_lap_month'] = test['maturity_lap_format'].dt.month

train['maturity_lap_day'] = train['maturity_lap_format'].dt.day
test['maturity_lap_day'] = test['maturity_lap_format'].dt.day

######################################################################

train['maturity_las_weekday'] = train['maturity_las_format'].dt.weekday
test['maturity_las_weekday'] = test['maturity_las_format'].dt.weekday

train['maturity_las_year'] = train['maturity_las_format'].dt.year
test['maturity_las_year'] = test['maturity_las_format'].dt.year

train['maturity_las_month'] = train['maturity_las_format'].dt.month
test['maturity_las_month'] = test['maturity_las_format'].dt.month

train['maturity_las_day'] = train['maturity_las_format'].dt.day
test['maturity_las_day'] = test['maturity_las_format'].dt.day

######################################################################


In [14]:
formatCols = [x for x in train.columns if  '_format' in x]
train.drop(formatCols, axis=1, inplace=True)
test.drop(formatCols, axis=1, inplace=True)

In [15]:
train['net_emi_paid'] = train['HL_EMI'] / train['SCRUB_EMI']
test['net_emi_paid'] = test['HL_EMI'] / test['SCRUB_EMI']

train['total_salary_credited'] = train['SAL_MON_01'] + train['SAL_MON_02'] + train['SAL_MON_03']
test['total_salary_credited'] = test['SAL_MON_01'] + test['SAL_MON_02'] + test['SAL_MON_03']

In [16]:
train['total_balance'] = train['EOP_MON_02'] + train['EOP_MON_03'] + train['EOP_BAL_MON_01']
test['total_balance'] =  test['EOP_MON_02'] + test['EOP_MON_03'] + test['EOP_BAL_MON_01']

In [17]:
cardCols = [x for x in train.columns if 'CARD_' in x]

for x in cardCols:
    train[x] = train[x].map(lambda x: 1 if x == 'Y' else 0)
    test[x] = test[x].map(lambda x: 1 if x == 'Y' else 0)

In [18]:
train['spent_sum'] = train[cardCols].sum(axis=1)
test['spent_sum'] = test[cardCols].sum(axis=1)

In [19]:
dcount = [x for x in train.columns if 'D_COUNT' in x]
ccount = [x for x in train.columns if 'C_COUNT' in x]

train['debit_count'] = train[dcount].sum(axis=1)
train['credit_count'] = train[ccount].sum(axis=1)

test['debit_count'] = test[dcount].sum(axis=1)
test['credit_count'] = test[ccount].sum(axis=1)

In [20]:
damount = [x for x in train.columns if 'D_AMT' in x]
camount = [x for x in train.columns if 'C_AMT' in x]

train['debit_amount'] = train[damount].sum(axis=1)
train['credit_amount'] = train[camount].sum(axis=1)

test['debit_amount'] = test[damount].sum(axis=1)
test['credit_amount'] = test[camount].sum(axis=1)

In [21]:
train['DESIGNATION_FINAL'] = train['DESIGNATION_FINAL'].map(lambda x: np.nan if x == 'MISSING' else x)
test['DESIGNATION_FINAL'] = test['DESIGNATION_FINAL'].map(lambda x: np.nan if x == 'MISSING' else x)

In [22]:
dest_tr = pd.get_dummies(train['DESIGNATION_FINAL'])
det_te = pd.get_dummies(test['DESIGNATION_FINAL'])

In [23]:
train = pd.concat([train, dest_tr], axis=1)
test = pd.concat([test, det_te], axis=1)

In [24]:
train.drop('DESIGNATION_FINAL', axis=1, inplace=True)
test.drop('DESIGNATION_FINAL', axis=1, inplace=True)

In [25]:
ccspend = sorted([x for x in train.columns if 'CC_SPEND' in x])
dcspend = sorted([x for x in train.columns if 'DC_SPEND' in x])

In [26]:
train['credit_spent'] = train['CC_SPEND_MON_01'] + train['CC_SPEND_MON_02'] + train['CC_SPEND_MON_03'] +train['CC_SPEND_MON_04'] + train['CC_SPEND_MON_05'] + train['CC_SPEND_MON_06'] 
test['credit_spent'] = test['CC_SPEND_MON_01'] + test['CC_SPEND_MON_02'] + test['CC_SPEND_MON_03'] + test['CC_SPEND_MON_04'] + test['CC_SPEND_MON_05'] + test['CC_SPEND_MON_06'] 

train['debit_spent'] = train['DC_SPEND_MON_01'] + train['DC_SPEND_MON_02'] + train['DC_SPEND_MON_03'] +train['DC_SPEND_MON_04'] + train['DC_SPEND_MON_05'] + train['DC_SPEND_MON_06'] 
test['debit_spent'] = test['DC_SPEND_MON_01'] + test['DC_SPEND_MON_02'] + test['DC_SPEND_MON_03'] +test['DC_SPEND_MON_04'] + test['DC_SPEND_MON_05'] + test['DC_SPEND_MON_06'] 

In [27]:
cats= ['_ATM', '_CARE', '_ENT', '_HBY', '_HMD', '_HTL', '_JER', '_MED', '_RST', '_TRL']

In [28]:
## total sum by categories
for i in ccspend:
    for j in cats:
        p = []
        if i.endswith(j):
            p.append(i)
        train['cc_sum_'+j] = train[p].sum(axis=1)
        test['cc_sum_'+j] = test[p].sum(axis=1)

In [29]:
## total sum by debit category
for i in dcspend:
    for j in cats:
        p = []
        if i.endswith(j):
            p.append(i)
        train['dc_sum_'+j] = train[p].sum(axis=1)
        test['dc_sum_'+j] = test[p].sum(axis=1)

In [30]:
## total credit spent is greater than credit limit

train['is_exceed_credit'] = train.apply(lambda row: 1 if (row['STMT_MON_01'] + row['STMT_MON_02'] + row['STMT_MON_03']) > row['CR_LIMIT'] else 0, axis=1)
test['is_exceed_credit'] = test.apply(lambda row: 1 if (row['STMT_MON_01'] + row['STMT_MON_02'] + row['STMT_MON_03']) > row['CR_LIMIT'] else 0, axis =1)

In [31]:
## count live, closed loans
liveLoan = [x for x in train.columns if 'SCRUB_LIVE' in x]
closedLoan = [x for x in train.columns if 'SCRUB_CLOSED' in x]

train['live_loan_count'] = train[liveLoan].sum(axis=1)
train['closed_loan_count'] = train[closedLoan].sum(axis=1)

test['live_loan_count'] = test[liveLoan].sum(axis=1)
test['closed_loan_count'] = test[closedLoan].sum(axis=1)

In [32]:
txn_mon = [x for x in train.columns if 'TXN_MON' in x]
txn_mon = txn_mon[:-1]

In [33]:
for c in txn_mon:
    p = []
    if 'CC_TXN' in c:
        p.append(c)
    train['total_credit_tran'] = train[p].sum(axis=1)
    test['total_credit_tran'] = test[p].sum(axis=1)
    
for k in txn_mon:
    j = []
    if 'DC_TXN' in k:
        j.append(c)
    train['total_debit_tran'] = train[j].sum(axis=1)
    test['total_debit_tran'] = test[j].sum(axis=1)

In [34]:
print(train.shape)
print(test.shape)

(300000, 391)
(200000, 390)


### one hot encoding - no one hot encoding this time

In [35]:
objCols = train.select_dtypes(include=['object']).columns

In [36]:
countDict = {}
for x in objCols:
    if train[x].nunique() > 2:
        countDict[x] = train[x].nunique()
        
oneHotCols = list(countDict.keys())

In [37]:
## check if object columns in both datasets have same unique values

for x in oneHotCols:
    if train[x].nunique() != test[x].nunique():
        set_train = set(train[x].unique())
        set_test = set(test[x].unique())
        
        remove_train = set_train - set_test
        remove_test = set_test - set_train
        
        remove = remove_train.union(remove_test)
        
        def filter_cat(x):
            if x in remove:
                return np.nan
            return x
        
        train[x] = train[x].map(filter_cat)
        test[x] = test[x].map(filter_cat)

In [38]:
import gc
gc.collect()

0

In [39]:
print(train.shape)
print(test.shape)

(300000, 391)
(200000, 390)


### label encoding

In [40]:
for x in train.columns:
    if x not in test.columns:
        print(x)

RESPONDERS


In [41]:
for x in test.columns:
    if x not in train.columns:
        print(x)

### remove columns with missing values - then label encode

In [44]:
misCols = train.isnull().sum(axis=0)/train.shape[0]
toremove = []

for k, v in misCols.iteritems():
    if v == 1:
        toremove.append(k)

In [45]:
len(toremove)

7

In [46]:
train.drop('OCCUP_ALL_NEW', axis=1, inplace=True)
test.drop('OCCUP_ALL_NEW', axis=1, inplace=True)

In [47]:
train.drop(toremove, inplace=True, axis=1)
test.drop(toremove, inplace=True, axis=1)

In [51]:
train.head()

,CUSTOMER_ID,ACT_TYPE,GENDER,LEGAL_ENTITY,AGE,N_CASA_MAX_BALANCE_MTD,N_CASA_MIN_BALANCE_MTD,BL_TAG,TWL_TAG,PL_TAG,AL_CNC_TAG,AGRI_TAG,CE_TAG,CV_TAG,EDU_TAG,TL_TAG,OTHER_LOANS_TAG,LAP_TAG,LAS_TAG,BBG_TAG,HL_TAG,GL_TAG,AL_CLOSED,PL_CLOSED,BL_CLOSED,TWL_CLOSED,CV_CLOSED,LAP_CLOSED,LAS_CLOSED,BBG_CLOSED,GL_CLOSED,CC_HOLD_MON_03,CC_HOLD_MON_02,CC_HOLD_MON_01,CC_ACTIVE_MON_03,CC_ACTIVE_MON_02,CC_ACTIVE_MON_01,CC_HOLD,CC_ACTIVE,DC_HOLD,DC_ACTIVE_MON_03,DC_ACTIVE_MON_02,DC_ACTIVE_MON_01,DC_ACTIVE,DC_SPEND_MON_03,DC_SPEND_MON_02,DC_SPEND_MON_01,CC_SPEND_MON_04,CC_SPEND_MON_01,CC_SPEND_MON_02,CC_SPEND_MON_03,DC_SPEND_MON_04,PM_FD_MON_01,CR_LIMIT,STMT_MON_03,STMT_MON_02,STMT_MON_01,COC_ELIGIBLE,LAP_SCRUB_LIVE,LAS_SCRUB_LIVE,PL_SCRUB_LIVE,GL_SCRUB_LIVE,TWL_SCRUB_LIVE,BL_SCRUB_LIVE,CEL_SCRUB_LIVE,TL_SCRUB_LIVE,AL_SCRUB_LIVE,LAP_SCRUB_CLOSED,LAS_SCRUB_CLOSED,PL_SCRUB_CLOSED,GL_SCRUB_CLOSED,TWL_SCRUB_CLOSED,BL_SCRUB_CLOSED,CEL_SCRUB_CLOSED,TL_SCRUB_CLOSED,AL_SCRUB_CLOSED,BTE_ACTIVE_MON_01,COC_ACTIVE_MON_01,PL2CC_ACTIVE_MON_01,MER_EMI_ACTIVE_MON_01,TRN_CON_DAE_ACTIVE_MON_01,BTE_CLOSED_MON_01,COC_CLOSED_MON_01,PL2CC_CLOSED_MON_01,TRN_CON_DAE_CLOSED_MON_01,EOP_BAL_MON_01,AMB_MON_04,AMB_MON_01,AMB_MON_02,AMB_MON_03,NB_MON_01_CNT,NB_MON_02_CNT,NB_MON_03_CNT,CDMAT_MON_01,CDMAT_MON_04,MF_MON_04,MF_MON_01,RD_MON_04,RD_MON_01,FD_MON_04,FD_MON_01,LI_MON_04,LI_MON_01,GI_MON_04,GI_MON_01,CC_TXN_MON_01,CC_SPEND_MON_06,CC_TXN_MON_06,CC_TXN_MON_03,CC_TXN_MON_02,CC_TXN_MON_04,CC_SPEND_MON_05,CC_TXN_MON_05,DC_TXN_MON_04,DC_TXN_MON_01,DC_SPEND_MON_06,DC_TXN_MON_06,DC_SPEND_MON_05,DC_TXN_MON_05,DC_TXN_MON_03,DC_TXN_MON_02,DC_SPEND_MON_04_ENT,DC_SPEND_MON_04_MED,DC_SPEND_MON_04_GRC,DC_SPEND_MON_04_HMD,DC_SPEND_MON_04_HBY,DC_SPEND_MON_04_CARE,DC_SPEND_MON_04_TRL,DC_SPEND_MON_04_RST,DC_SPEND_MON_04_JER,DC_SPEND_MON_04_HTL,DC_SPEND_MON_04_ATM,DC_SPEND_MON_03_ENT,DC_SPEND_MON_03_MED,DC_SPEND_MON_03_HMD,DC_SPEND_MON_03_HBY,DC_SPEND_MON_03_CARE,DC_SPEND_MON_03_TRL,DC_SPEND_MON_03_RST,DC_SPEND_MON_03_JER,DC_SPEND_MON_03_HTL,DC_SPEND_MON_03_ATM,DC_SPEND_MON_02_ENT,DC_SPEND_MON_02_MED,DC_SPEND_MON_02_HMD,DC_SPEND_MON_02_HBY,DC_SPEND_MON_02_CARE,DC_SPEND_MON_02_TRL,DC_SPEND_MON_02_RST,DC_SPEND_MON_02_JER,DC_SPEND_MON_02_HTL,DC_SPEND_MON_02_ATM,DC_SPEND_MON_01_ENT,DC_SPEND_MON_01_MED,DC_SPEND_MON_01_HMD,DC_SPEND_MON_01_HBY,DC_SPEND_MON_01_CARE,DC_SPEND_MON_01_TRL,DC_SPEND_MON_01_RST,DC_SPEND_MON_01_JER,DC_SPEND_MON_01_HTL,DC_SPEND_MON_01_ATM,DC_SPEND_MON_05_ENT,DC_SPEND_MON_05_MED,DC_SPEND_MON_05_HMD,DC_SPEND_MON_05_HBY,DC_SPEND_MON_05_CARE,DC_SPEND_MON_05_TRL,DC_SPEND_MON_05_RST,DC_SPEND_MON_05_JER,DC_SPEND_MON_05_HTL,DC_SPEND_MON_05_ATM,CC_SPEND_MON_04_ENT,CC_SPEND_MON_04_MED,CC_SPEND_MON_04_HMD,CC_SPEND_MON_04_HBY,CC_SPEND_MON_04_CARE,CC_SPEND_MON_04_TRL,CC_SPEND_MON_04_RST,CC_SPEND_MON_04_JER,CC_SPEND_MON_04_HTL,CC_SPEND_MON_04_ATM,CC_SPEND_MON_02_ENT,CC_SPEND_MON_02_MED,CC_SPEND_MON_02_HMD,CC_SPEND_MON_02_HBY,CC_SPEND_MON_02_CARE,CC_SPEND_MON_02_TRL,CC_SPEND_MON_02_RST,CC_SPEND_MON_02_JER,CC_SPEND_MON_02_HTL,CC_SPEND_MON_02_ATM,CC_SPEND_MON_01_ENT,CC_SPEND_MON_01_MED,CC_SPEND_MON_01_HMD,CC_SPEND_MON_01_HBY,CC_SPEND_MON_01_CARE,CC_SPEND_MON_01_TRL,CC_SPEND_MON_01_RST,CC_SPEND_MON_01_JER,CC_SPEND_MON_01_HTL,CC_SPEND_MON_01_ATM,FINANCE_MON_01,JOBS_MON_01,LIFESTYLE_MON_01,INMON_01KET_MON_01,TRAVEL_MON_01,CHANNEL_CLICK_DISP,TOP14_CITY,TOP9_CITY,PL2CC_ELIGIBLE,D_AMT_L3_MON_06,C_AMT_L3_MON_06,C_COUNT_L3_MON_06,D_COUNT_L3_MON_06,MAX_C_AMT_L3_MON_06,D_AMT_L3_MON_05,C_AMT_L3_MON_05,C_COUNT_L3_MON_05,D_COUNT_L3_MON_05,MAX_C_AMT_L3_MON_05,D_AMT_L3_MON_04,C_AMT_L3_MON_04,C_COUNT_L3_MON_04,D_COUNT_L3_MON_04,MAX_C_AMT_L3_MON_04,D_AMT_L3_MON_03,C_AMT_L3_MON_03,C_COUNT_L3_MON_03,D_COUNT_L3_MON_03,MAX_C_AMT_L3_MON_03,D_AMT_L3_MON_02,C_AMT_L3_MON_02,C_COUNT_L3_MON_02,D_COUNT_L3_MON_02,MAX_C_AMT_L3_MON_02,D_AMT_L3_MON_01,C_AMT_L3_MON_01,C_COUNT_L3_MON_01,D_COUNT_L3_MON_01,MAX_C_AMT_L3_MON_01,PA_PQ_TAG,NEFT_CC_AMOUNT,NEFT_CC_TXN,NEFT_CC_CATEGORY,NEFT_DC_AMOUNT,NEFT_DC_TXN,NEFT_DC_CATEGOR

In [53]:
train.dtypes.value_counts()

float64    278
object      51
int64       44
uint8       10
dtype: int64

In [55]:
colsint = train.select_dtypes(include=['uint8']).columns

In [60]:
for c in train.select_dtypes(include=['object']).columns:
    if c not in ['CUSTOMER_ID','RESPONDERS'] + colsint.tolist():
        print(c)
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values) + list(test[c].values))
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))

GL_CLOSED
CC_HOLD_MON_03
CC_HOLD_MON_02
CC_HOLD_MON_01
CC_ACTIVE_MON_03
CC_ACTIVE_MON_02
CC_ACTIVE_MON_01
CC_HOLD
CC_ACTIVE
DC_HOLD
DC_ACTIVE_MON_03
DC_ACTIVE_MON_02
DC_ACTIVE_MON_01
DC_ACTIVE
BTE_ACTIVE_MON_01
COC_ACTIVE_MON_01
PL2CC_ACTIVE_MON_01
MER_EMI_ACTIVE_MON_01
TRN_CON_DAE_ACTIVE_MON_01
BTE_CLOSED_MON_01
COC_CLOSED_MON_01
PL2CC_CLOSED_MON_01
TRN_CON_DAE_CLOSED_MON_01
FINANCE_MON_01
JOBS_MON_01
LIFESTYLE_MON_01
INMON_01KET_MON_01
TRAVEL_MON_01
CHANNEL_CLICK_DISP
TOP14_CITY
TOP9_CITY
PA_PQ_TAG
NEFT_CC_CATEGORY
NEFT_DC_CATEGORY
TPT_DC_CATEGORY_MON_01
TPT_CC_CATEGORY_MON_01
IMPS_CC_CATEGORY_MON_01
ZIP_CODE_FINAL
LAST_3MTHS_INQ
PREV_3MTHS_INQ
TOP_100_CORP
TOP_800_CORP
TOP_CORP_TAG
CREDIT_ACT
DEBIT_ACT
CEL_TAG
EL_TAG
AGL_TAG
CVL_TAG
AL_TAG


In [61]:
train.fillna(-1, inplace=True)
test.fillna(-1, inplace=True)

In [74]:
for x in colsint:
    train[x] = train[x].astype(np.int64)
    test[x] = test[x].astype(np.int64)

In [85]:
## remove outliers

for x in train.columns:
    if train[x].dtype == np.float64:
        ulimit = np.percentile(train[x].values, 99.5)
        llimit = np.percentile(train[x].values, 0.5)
        train[x].ix[train[x]>ulimit] = ulimit
        train[x].ix[train[x]<llimit] = llimit
        
        ulimit2 = np.percentile(test[x].values, 99.5)
        llimit2 = np.percentile(test[x].values, 0.5)
        test[x].ix[test[x]>ulimit2] = ulimit2
        test[x].ix[test[x]<llimit2] = llimit2

/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


### Do PCA and get new features

In [86]:
feature_names = [x for x in train.columns if x not in ['CUSTOMER_ID','RESPONDERS']]

In [87]:
dim_train = train[feature_names]
dim_test = test[feature_names]

In [88]:
from sklearn.preprocessing import MinMaxScaler

In [89]:
rb = MinMaxScaler()

In [90]:
scaler = rb.fit(dim_train)

In [91]:
dim_train = scaler.transform(dim_train)
dim_test = scaler.transform(dim_test)

In [92]:
n_comp = 10

In [93]:
tsvd = TruncatedSVD(n_components=n_comp, random_state=42)
tsvd_train = tsvd.fit_transform(dim_train)
tsvd_test = tsvd.transform(dim_test)

In [94]:
pca = PCA(n_components=n_comp, random_state=420)
pca_train = pca.fit_transform(dim_train)
pca_test = pca.transform(dim_test)

In [95]:
ica = FastICA(n_components=n_comp, random_state=2030)
ica_train = ica.fit_transform(dim_train)
ica_test = ica.transform(dim_test)

In [96]:
grp = GaussianRandomProjection(n_components=n_comp, random_state=42)
grp_train = grp.fit_transform(dim_train)
grp_test = grp.transform(dim_test)

In [97]:
srp = SparseRandomProjection(n_components=n_comp, random_state=42)
srp_train = srp.fit_transform(dim_train)
srp_test = srp.transform(dim_test)

In [98]:
for i in range(1, n_comp + 1):
    train['pca_' + str(i)] = pca_train[:,i-1]
    test['pca_' + str(i)] = pca_test[:,i-1]
    
    train['tsvd_' + str(i)] = tsvd_train[:,i-1]
    test['tsvd_' + str(i)] = tsvd_test[:,i-1]
    
    train['ica_' + str(i)] = ica_train[:,i-1]
    test['ica_' + str(i)] = ica_test[:,i-1]
    
    train['grp_' + str(i)] = grp_train[:,i-1]
    test['grp_' + str(i)] = grp_test[:,i-1]
    
    train['srp_' + str(i)] = srp_train[:,i-1]
    test['srp_' + str(i)] = srp_test[:,i-1]
    
#     train['tsne_' + str(i)] = tsne_train[:,i-1]
#     test['tsne_' + str(i)] = tsne_test[:,i-1]

In [99]:
del dim_train, dim_test

In [100]:
print(train.shape)
print(test.shape)

(300000, 433)
(200000, 432)


### Load Level 1 Preds

In [101]:
et_train = pd.read_csv('submissions/oof_preds/et_train.csv')
et_test = pd.read_csv('submissions/oof_preds/et_test.csv')

lgb_train = pd.read_csv('submissions/oof_preds/lgb_train1.csv')
lgb_test = pd.read_csv('submissions/oof_preds/lgb_test1.csv')

lr_train = pd.read_csv('submissions/oof_preds/xgb_LR_train.csv')
lr_test = pd.read_csv('submissions/oof_preds/xgb_LR_test.csv')

xg_train = pd.read_csv('submissions/oof_preds/xgb_XG_train.csv')
xg_test = pd.read_csv('submissions/oof_preds/xgb_XG_test.csv')

In [102]:
train['et_pred'] = et_train['RESPONDERS']
test['et_pred'] = et_test['RESPONDERS']

train['lgb_pred'] = lgb_train['RESPONDERS']
test['lgb_pred'] = lgb_test['RESPONDERS']

train['lr_pred'] = lr_train['RESPONDERS']
test['lr_pred'] = lr_test['RESPONDERS']

train['xg_pred'] = xg_train['RESPONDERS']
test['xg_pred'] = xg_test['RESPONDERS']

In [103]:
del et_train, et_test, lgb_train, lgb_test, lr_train, lr_test, xg_train, xg_test

In [104]:
print(train.shape)
print(test.shape)

(300000, 437)
(200000, 436)


##  Single XGB Model

### XGB 1

In [105]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [106]:
target = train['RESPONDERS']
target = target.map(lambda x: 1 if x == 'Y' else 0)

In [107]:
feature_names = [x for x in train.columns if x not in ['CUSTOMER_ID','RESPONDERS']]

In [108]:
X_train, X_valid, y_train, y_valid = train_test_split(train, target, test_size = 0.2, stratify = target)

In [109]:
params = {"objective": "binary:logistic",
          "booster": "gbtree",
          "nthread": 4,
          "eta": 0.1, # 0.1
          "max_depth": 5, # 7
          "subsample": 0.8,
          "colsample_bytree": 0.4,
          "min_child_weight": 2**3,
          "seed": 2016, 
          "tree_method": "exact",
         "eval_metric":"auc"}

In [110]:
## normalised data
dtrain = xgb.DMatrix(X_train[feature_names], y_train, missing=-1)
dvalid = xgb.DMatrix(X_valid[feature_names], y_valid, missing=-1)
dtest = xgb.DMatrix(test[feature_names], missing=-1)

In [111]:
watchlist = [(dtrain, 'train'),(dvalid, 'valid')]

In [112]:
clf1 = xgb.train(params, dtrain, num_boost_round=1000, evals=watchlist, maximize=True, verbose_eval=20, early_stopping_rounds=40)

[0]	train-auc:0.889137	valid-auc:0.878793
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 40 rounds.
[20]	train-auc:0.902909	valid-auc:0.88871
[40]	train-auc:0.910611	valid-auc:0.891923
[60]	train-auc:0.918646	valid-auc:0.893932
[80]	train-auc:0.926169	valid-auc:0.895462
[100]	train-auc:0.930763	valid-auc:0.895339
[120]	train-auc:0.93403	valid-auc:0.895708
[140]	train-auc:0.937454	valid-auc:0.895367
Stopping. Best iteration:
[118]	train-auc:0.933683	valid-auc:0.895949



In [ ]:
## important features sorted
imp_feats = sorted(clf1.get_fscore(), key=lambda x: x[1], reverse=True)

In [113]:
preds1 = clf1.predict(dtest)

In [114]:
sub = pd.read_csv('sample_submission_ROqqAAN.csv')
sub['CUSTOMER_ID'] = test['CUSTOMER_ID']
sub['RESPONDERS'] = preds1

In [115]:
sub.to_csv('submissions/stack_six.csv', index=False) ## 0.68

## export data for R Modeling

In [120]:
train.to_csv('submissions/train_final.csv', index=False)
test.to_csv('submissions/test_final.csv', index=False)

### XGB with imp. features

In [ ]:
len(imp_feats)

In [ ]:
top_250 = imp_feats[:250]

In [ ]:
top_350 = imp_feats[:350] ## no improvement

In [ ]:
dtrain = xgb.DMatrix(X_train[top_250], y_train, missing=-1)
dvalid = xgb.DMatrix(X_valid[top_250], y_valid, missing=-1)
dtest = xgb.DMatrix(test[top_250], missing=-1)

In [ ]:
params = {"objective": "binary:logistic",
          "booster": "gbtree",
          "nthread": 4,
          "eta": 0.1, # 0.1
          "max_depth": 5, # 7
          "subsample": 0.8,
          "colsample_bytree": 0.8,
          "min_child_weight": 2,
          "seed": 2016, 
          "tree_method": "exact",
         "eval_metric":"auc"}

In [ ]:
tuned_params = {"objective": "binary:logistic",
          "booster": "gbtree",
          "nthread": 4,
          "eta": 0.1, # 0.1
          "max_depth": 6, # 7
          "subsample": 1,
          "colsample_bytree": 1,
          "min_child_weight": 10,
          "seed": 2017, 
          "alpha": 2**2,
          #"tree_method": "exact",
         "eval_metric":"auc"}

In [ ]:
watchlist = [(dtrain, 'train'),(dvalid, 'valid')]

In [ ]:
clf1 = xgb.train(tuned_params, dtrain, num_boost_round=10000, evals=watchlist, maximize=True, verbose_eval=20, early_stopping_rounds=40)

In [ ]:
preds1 = clf1.predict(dtest)
sub = pd.read_csv('sample_submission_ROqqAAN.csv')
sub['CUSTOMER_ID'] = test['CUSTOMER_ID']
sub['RESPONDERS'] = preds1
sub.to_csv('submissions/stack_two.csv', index=False)

# By removing features, AUC has increased slightly. LB improved # ## 0.6775

In [ ]:
preds1 = clf1.predict(dtest)
sub = pd.read_csv('sample_submission_ROqqAAN.csv')
sub['CUSTOMER_ID'] = test['CUSTOMER_ID']
sub['RESPONDERS'] = preds1
sub.to_csv('submissions/stack_two.csv', index=False)


### XGB with Logistic Regression - top 250

In [ ]:
## normalised data
dtrain = xgb.DMatrix(X_train[top_250], y_train)
dvalid = xgb.DMatrix(X_valid[top_250], y_valid)
dtest = xgb.DMatrix(test[top_250])

In [ ]:
params = {"objective": "binary:logistic",
          "booster": "gblinear",
          "nthread": 4,
          "alpha": 2**3,
          "lambda": 2**4,
         "eval_metric":"auc"}

In [ ]:
watchlist = [(dtrain, 'train'),(dvalid, 'valid')]

In [ ]:
clf1 = xgb.train(params, dtrain, num_boost_round=10000, evals=watchlist, maximize=True, verbose_eval=20, early_stopping_rounds=40)

In [ ]:
preds1 = clf1.predict(dtest)
sub = pd.read_csv('sample_submission_ROqqAAN.csv')
sub['CUSTOMER_ID'] = test['CUSTOMER_ID']
sub['RESPONDERS'] = preds1

In [ ]:
sub.to_csv('submissions/stack_three.csv', index=False) ## 0.10

### XGB SINGLE MODEL WITH WEIGHTS

### XGB 2

In [ ]:
train['weights'] = train['RESPONDERS'].map(lambda x: 0.8 if x == 1 else 0)

In [ ]:
feature_names2 = [x for x in train.columns if x not in ['CUSTOMER_ID','RESPONDERS']]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train, target, test_size = 0.3, stratify = target)

In [ ]:
params = {"objective": "binary:logistic",
          "booster": "gbtree",
          "nthread": 4,
          "eta": 0.1, # 0.1
          "max_depth": 5, # 7
          "subsample": 0.8,
          "colsample_bytree": 0.4,
          "min_child_weight": 2**3,
          "seed": 2016, 
          "tree_method": "exact",
         "eval_metric":"auc"}

In [ ]:
dtrain = xgb.DMatrix(X_train[feature_names2], y_train, weight=weights)
dvalid = xgb.DMatrix(X_valid[feature_names2], y_valid, weight=weights)
dtest = xgb.DMatrix(test[feature_names])

In [ ]:
watchlist = [(dtrain, 'train'),(dvalid, 'valid')]

In [ ]:
clf1 = xgb.train(params, dtrain, num_boost_round=10000, evals=watchlist, maximize=True, verbose_eval=20, early_stopping_rounds=40)

default params are giving lower score than tuned

### EXTRA TREES

In [ ]:
train.drop('weights', axis=1, inplace=True)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, make_scorer

In [ ]:
et = ExtraTreesClassifier(n_estimators=100, 
                          criterion = 'entropy', 
                          max_depth=10, 
                          max_features='auto',
                          min_samples_split = 3,
                          n_jobs = -1,
                         random_state = 4343)
                         #class_weight = {0:1, 1:10})

In [ ]:
feature_names2 = [x for x in feature_names2 if 'weights' not in x]

In [ ]:
cross_val_score(et, train[feature_names2], target, scoring=make_scorer(roc_auc_score), cv = 5)

### XGB Random Forest

In [ ]:
param_dart = {'booster': 'dart',
         'max_depth': 5, 'learning_rate': 0.1,
         'objective': 'binary:logistic',
            'sample_type': 'uniform',
         'normalize_type': 'tree',
         'rate_drop': 0.1,
         'skip_drop': 0.5,
             'eval_metric':'auc'}

In [ ]:
param_forest = {'booster': 'dart',
         'max_depth': 5, 'learning_rate': 0.1,
         'objective': 'binary:logistic',
            'sample_type': 'uniform',
         'normalize_type': 'forest',
         'rate_drop': 0.2,
         'skip_drop': 0.6,
             'eval_metric':'auc'}

In [ ]:
#dtrain = xgb.DMatrix(X_train[feature_names], y_train, missing=-999)
#dvalid = xgb.DMatrix(X_valid[feature_names], y_valid, missing=-999)
#dtest = xgb.DMatrix(test[feature_names], missing=-999)

## normalised data
dtrain = xgb.DMatrix(X_train[feature_names], y_train)
dvalid = xgb.DMatrix(X_valid[feature_names], y_valid)
dtest = xgb.DMatrix(test[feature_names])

In [ ]:
watchlist = [(dtrain, 'train'),(dvalid, 'valid')]

In [ ]:
clf1 = xgb.train(param_forest, dtrain, num_boost_round=1000, evals=watchlist, maximize=True, verbose_eval=20, early_stopping_rounds=40)

In [ ]:
preds1 = clf1.predict(dtest)
sub = pd.read_csv('sample_submission_ROqqAAN.csv')
sub['CUSTOMER_ID'] = test['CUSTOMER_ID']
sub['RESPONDERS'] = preds1

In [ ]:
sub.to_csv('submissions/xgb_randomforest.csv', index=False) ## CV - 89.3244 | LB 0.651 ## type == 'tree'

In [ ]:
sub.to_csv('submissions/xgb_randomforest_forest.csv', index=False) ## CV - 89.3427 | LB 0.64629 ## type == 'forest'

## Stacker - Random Forest - with scaled data

In [ ]:
nfolds = 10
skf = StratifiedKFold(n_splits=nfolds, shuffle=True, random_state=9087)

In [ ]:
param_dart = {'booster': 'dart',
         'max_depth': 5, 'learning_rate': 0.1,
         'objective': 'binary:logistic',
            'sample_type': 'uniform',
         'normalize_type': 'tree',
         'rate_drop': 0.1,
         'skip_drop': 0.5,
             'eval_metric':'auc'}

In [ ]:
oof_train = pd.DataFrame({'CUSTOMER_ID':train['CUSTOMER_ID'], 'RESPONDERS':0})
allpredictions = pd.DataFrame()
score = []

In [ ]:
## without upscaling

increase = True
for i, (train_index, test_index) in enumerate(skf.split(train, target)):
    print('Fold %d/%d'%(i+1, nfolds))
    X_train, X_valid = train.iloc[train_index], train.iloc[test_index]
    y_train, y_valid = target.iloc[train_index], target.iloc[test_index]
    
#     if increase:
#         pos = pd.Series(target == 1)

#         X_train = pd.concat([X_train, X_train[pos]], axis=0)
#         y_train = pd.concat([y_train, y_train.loc[pos]], axis=0)
        
#         idx = np.arange(len(X_train))
#         np.random.shuffle(idx)
#         X_train = X_train.iloc[idx]
#         y_train = y_train.iloc[idx]
    
    dtrain = xgb.DMatrix(X_train[feature_names], label=y_train)
    dvalid = xgb.DMatrix(X_valid[feature_names], label=y_valid)
    
    watchlist = [(dtrain, 'train'),(dvalid,'valid')]
    
    clf = xgb.train(param_dart,
                    dtrain,
                    num_boost_round=10000,
                    evals=watchlist,
                    early_stopping_rounds=40,
                    verbose_eval=20,
                    maximize = True)
    
    pred1 = clf.predict(dvalid)
    oof_train.loc[test_index, 'RESPONDERS'] = pred1
    
    scr = roc_auc_score(y_valid, pred1)
    dtest = xgb.DMatrix(test[feature_names])
    preds2 = clf.predict(dtest)
    
    allpredictions['p'+str(i)] = preds2
    score.append(scr)
    
    del X_train, X_valid, y_train, y_valid, clf, pred1, preds2, scr

In [ ]:
np.mean(score)

In [ ]:
oof_train.to_csv('submissions/oof_preds/xgb_train1.csv', index=False)

In [ ]:
allpredictions.head()

In [ ]:
sub = pd.read_csv('sample_submission_ROqqAAN.csv')
sub['CUSTOMER_ID'] = test['CUSTOMER_ID']
sub['RESPONDERS'] = allpredictions.mean(axis=1).values

In [ ]:
sub.to_csv('submissions/xgb_bagging_seven.csv', index=False) # 0.58

In [ ]:
train.shape

In [ ]:
test.shape

### XGB 2 - remove zero variance features

In [ ]:
## remove zero variance columns
var_check = pd.DataFrame(columns=['name','variance'])

for i, c in enumerate(train.columns):
    if c not in ['CUSTOMER_ID','RESPONDERS']:
        var_check.loc[i, 'name'] = c
        var_check.loc[i, 'variance'] = train[c].var()
        
var_check.sort_values('variance', ascending=True).reset_index(drop=True)

In [ ]:
cols_to_use = var_check[var_check['variance'] > 1]['name'].tolist() ## remove columns with variance lesser than zero

In [ ]:
params = {"objective": "binary:logistic",
          "booster": "gbtree",
          "nthread": 4,
          "eta": 0.1,
          "max_depth": 5,
          "subsample": 0.8,
          "colsample_bytree": 0.3,
          "min_child_weight": 0,
          "seed": 2017, 
          "tree_method": "exact",
         "eval_metric":"auc"}

In [ ]:
dtrain = xgb.DMatrix(X_train[cols_to_use], y_train, missing=-1)
dvalid = xgb.DMatrix(X_valid[cols_to_use], y_valid, missing=-1)
dtest = xgb.DMatrix(test[cols_to_use], missing=-1)

In [ ]:
watchlist = [(dtrain, 'train'),(dvalid, 'valid')]

In [ ]:
clf1 = xgb.train(params, dtrain, num_boost_round=10000, evals=watchlist, maximize=True, verbose_eval=20, early_stopping_rounds=40)

In [ ]:
preds1 = clf1.predict(dtest)

In [ ]:
sub = pd.read_csv('sample_submission_ROqqAAN.csv')
sub['CUSTOMER_ID'] = test['CUSTOMER_ID']
sub['RESPONDERS'] = preds1

In [ ]:
sub.to_csv('submissions/xgb_single_three.csv', index=False) ## 54 - score dropping.

### XGB BAGGED

In [ ]:
oof_train = pd.DataFrame({'CUSTOMER_ID':train['CUSTOMER_ID'], 'RESPONDERS':0})
allpredictions = pd.DataFrame()
score = []

In [ ]:
from sklearn.model_selection import StratifiedKFold
nfolds = 7
skf = StratifiedKFold(n_splits=nfolds, random_state=42)

In [ ]:
increase = True
for i, (train_index, test_index) in enumerate(skf.split(train, target)):
    print('Fold %d/%d'%(i+1, nfolds))
    X_train, X_valid = train.iloc[train_index], train.iloc[test_index]
    y_train, y_valid = target.iloc[train_index], target.iloc[test_index]
    
    if increase:
        pos = pd.Series(target == 1)

        X_train = pd.concat([X_train, X_train[pos]], axis=0)
        y_train = pd.concat([y_train, y_train.loc[pos]], axis=0)
        
        idx = np.arange(len(X_train))
        np.random.shuffle(idx)
        X_train = X_train.iloc[idx]
        y_train = y_train.iloc[idx]
    
    dtrain = xgb.DMatrix(X_train[top_250], label=y_train, missing=-1)
    dvalid = xgb.DMatrix(X_valid[top_250], label=y_valid, missing = -1)
    
    watchlist = [(dtrain, 'train'),(dvalid,'valid')]
    
    clf = xgb.train(params,
                    dtrain,
                    num_boost_round=10000,
                    evals=watchlist,
                    early_stopping_rounds=40,
                    verbose_eval=20,
                    maximize = True)
    
    pred1 = clf.predict(dvalid)
    oof_train.loc[test_index, 'RESPONDERS'] = pred1
    
    scr = roc_auc_score(y_valid, pred1)
    dtest = xgb.DMatrix(test[top_250], missing= - 1)
    preds2 = clf.predict(dtest)
    
    allpredictions['p'+str(i)] = preds2
    score.append(scr)
    
    del X_train, X_valid, y_train, y_valid, clf, pred1

In [ ]:
sub = pd.read_csv('sample_submission_ROqqAAN.csv')
sub['CUSTOMER_ID'] = test['CUSTOMER_ID']
sub['RESPONDERS'] = allpredictions.mean(axis=1).values

In [ ]:
sub.to_csv('submissions/stack_bagged_four.csv', index=False)

## Features for Thanish

In [ ]:
imp_train = train[top_250]
imp_test = test[top_250]

In [ ]:
top_100 = imp_feats[:100]

In [ ]:
imp_train = train[top_100]
imp_test = test[top_100]

In [ ]:
imp_train.to_csv('submissions/train_100.csv', index=False)
imp_test.to_csv('submissions/test_100.csv', index=False)